In [3]:
import os
import re
import zipfile
from PyPDF2 import PdfReader

# 📂 Pasta onde estão os .zip baixados
pasta_zips = "/Users/thiagomoncinhatto/Documents/02_MBA_Financas/TCC/DFs/DFP_2024"

# 📂 Pasta destino para os PDFs
pasta_destino = os.path.join(pasta_zips, "DFPs_CVM")
os.makedirs(pasta_destino, exist_ok=True)

def normalizar_nome(nome):
    # Remove acentos, espaços, caracteres especiais → formato de arquivo
    nome = nome.lower()
    nome = re.sub(r"[áàãâä]", "a", nome)
    nome = re.sub(r"[éèêë]", "e", nome)
    nome = re.sub(r"[íìîï]", "i", nome)
    nome = re.sub(r"[óòõôö]", "o", nome)
    nome = re.sub(r"[úùûü]", "u", nome)
    nome = re.sub(r"ç", "c", nome)
    nome = re.sub(r"[^a-z0-9]+", "_", nome)
    nome = re.sub(r"_+", "_", nome).strip("_")
    return nome

def extrair_nome_empresa(pdf_path):
    try:
        reader = PdfReader(pdf_path)
        primeira_pagina = reader.pages[0].extract_text()
        if not primeira_pagina:
            return None
        
        # Procura pelo padrão (empresa geralmente vem após a data)
        match = re.search(r"DFP.*\d{2}/\d{2}/\d{4}\s*-\s*(.+)", primeira_pagina)
        if match:
            return match.group(1).strip()
    except Exception as e:
        print(f"⚠️ Erro ao ler PDF {pdf_path}: {e}")
    return None

# 🔄 Processa todos os zips
for arquivo in os.listdir(pasta_zips):
    if not arquivo.lower().endswith(".zip"):
        continue

    zip_path = os.path.join(pasta_zips, arquivo)
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            for pdf_nome in zip_ref.namelist():
                if not pdf_nome.lower().endswith(".pdf"):
                    continue
                
                # Extrai temporariamente
                temp_path = os.path.join(pasta_destino, "temp.pdf")
                with open(temp_path, "wb") as f:
                    f.write(zip_ref.read(pdf_nome))

                # Descobre o nome da empresa
                empresa = extrair_nome_empresa(temp_path)
                if empresa:
                    nome_final = normalizar_nome(empresa) + ".pdf"
                else:
                    nome_final = os.path.splitext(pdf_nome)[0] + ".pdf"

                destino_final = os.path.join(pasta_destino, nome_final)

                # Garante nomes únicos se houver conflito
                contador = 1
                base_nome = destino_final
                while os.path.exists(destino_final):
                    destino_final = base_nome.replace(".pdf", f"_{contador}.pdf")
                    contador += 1

                os.rename(temp_path, destino_final)
                print(f"✅ Extraído: {destino_final}")
    except Exception as e:
        print(f"❌ Erro processando {zip_path}: {e}")

✅ Extraído: /Users/thiagomoncinhatto/Documents/02_MBA_Financas/TCC/DFs/DFP_2024/DFPs_CVM/anemus_wind_holding_s_a_versao_2.pdf
✅ Extraído: /Users/thiagomoncinhatto/Documents/02_MBA_Financas/TCC/DFs/DFP_2024/DFPs_CVM/telec_brasileiras_s_a_telebras_versao_1.pdf
✅ Extraído: /Users/thiagomoncinhatto/Documents/02_MBA_Financas/TCC/DFs/DFP_2024/DFPs_CVM/travessia_securitizadora_s_a_versao_3.pdf
✅ Extraído: /Users/thiagomoncinhatto/Documents/02_MBA_Financas/TCC/DFs/DFP_2024/DFPs_CVM/companhia_itaunense_energia_e_participacoes_versao_1.pdf
✅ Extraído: /Users/thiagomoncinhatto/Documents/02_MBA_Financas/TCC/DFs/DFP_2024/DFPs_CVM/trevisa_investimentos_s_a_versao_1.pdf
✅ Extraído: /Users/thiagomoncinhatto/Documents/02_MBA_Financas/TCC/DFs/DFP_2024/DFPs_CVM/clear_sale_s_a_versao_1.pdf
✅ Extraído: /Users/thiagomoncinhatto/Documents/02_MBA_Financas/TCC/DFs/DFP_2024/DFPs_CVM/transmissora_jose_maria_de_macedo_de_eletricidade_s_a_versao_1.pdf
✅ Extraído: /Users/thiagomoncinhatto/Documents/02_MBA_Financas/